# Merge data of each team

This Python script for analyzing and cleaning WAR (Wins Above Replacement) data of MLB players during the 2024 regular season. The data was obtained from Baseball Reference.

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
fname = "2024_Player_Stats_MLB.xlsx"

xls = pd.ExcelFile(fname)
sheets = xls.sheet_names

ds = {}

for sheet in sheets:
    if sheet == 'Team':
        team = pd.read_excel(fname, sheet_name=sheet)
    else:
        open = pd.read_excel(fname, sheet_name=sheet)
        open.loc[:, 'Team'] = sheet.split('_')[0]
        # add the classification of Pitcher and Batter
        if '_P' in sheet:
            open.loc[:, 'Func'] = "Pitching"
        else:
            open.loc[:, 'Func'] = "Batting"

        
        ds[sheet] = open.copy()

ds = pd.concat([dat for _, dat in ds.items()], ignore_index=True)

In [ ]:
# list of MLB teams
team_abb = team['Abbrevation'].to_list()

In [ ]:
dat = ds.copy()

In [ ]:
# clean the data and also add data order (Y)

data = {}

for t in team_abb:

    mask = (dat['Team'] == t)
    copy = dat.loc[mask].copy()
    copy = copy.sort_values(by='Player').reset_index(drop=True)
    copy.loc[:, 'Order'] = [x for x in range(1, dat.loc[mask].shape[0]+1)] 
    copy.loc[:, 'Player Name'] = copy['Player'].str.replace("#", "").str.replace("*", "")
    copy.loc[:, 'Y'] = copy.groupby('WAR').cumcount() - (copy.groupby('WAR')['WAR'].transform('size') // 2)
    data[t] = copy.copy()

data = pd.concat([dat for _, dat in data.items()], ignore_index=True)

In [ ]:
# calculate whether a player is qualified for stats ranking (courtesy of MLB rules)
def calculate_qualification(pos, app):
    fielding = ['1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF','UT', 'CI', 'IF', 'OF']

    if pos == 'DH':
        return 'Qualified' if app >= 3.1 else "Not Qualified" 

    elif pos in fielding :
        return 'Qualified' if app >= 0.67 else "Not Qualified" 
    elif pos == 'C':
        if app >= 0.5:
            return 'Qualified'
        else:
            return "Not Qualified"
        
    elif pos == 'RP':
        return 'Qualified' if app >= 0.5 else "Not Qualified" 
    
    elif pos in ['SP', 'CL']:
        return 'Qualified' if app >= 1.0 else "Not Qualified" 

    else:
        return "Not Qualified"

In [ ]:
# calculate appearances per game
# PA = plate appearances (for batter) and IP = inning pitching
data.loc[:, 'APP'] = np.where(
    data['Func'] == 'Batting',
    data['PA'] / data['G'],
    data['IP'] / data['G'])

In [ ]:
data.loc[:, 'Qualified'] = data.apply(lambda row: calculate_qualification(row.get('Pos'), row['APP']), axis=1)

In [ ]:
# split the awards column into 6 awards category
keys= ['GG', 'SS', 'AS', 'MVP', 'CYA', 'ROY'] # category

for key in keys:
    if key in ['GG', 'SS', 'AS']:
        data.loc[:, key] = np.where(data['Awards'].str.contains(key).replace({None: False}), key, None)
    else:
        data.loc[:, key] = data['Awards'].str.extract(fr'({key}-\d+)')[0]

In [ ]:
# combine all the awards into single column
def awards(row):
    awards_list = []
    keys= ['GG', 'SS', 'AS', 'MVP', 'CYA', 'ROY']
    for k in keys:
        if row[k] != "nan":
            awards_list.append(row[k])
        else:
            continue
    
    return " / ".join(awards_list)    

In [ ]:
import numpy as np

keys = ['GG', 'SS', 'AS', 'MVP', 'CYA', 'ROY']
for k in keys:
    data.loc[:, k] = data[k].replace({np.nan:"0"}).astype(str)
    print(data[k].unique())

In [ ]:
data.loc[:,'Awards List'] = data.apply(lambda row: awards(row) , axis=1)

In [ ]:
data.to_excel('WAR_MLB_2024.xlsx', index=False)